In [1]:
import subprocess
from distutils.version import LooseVersion

import dxdata
import dxpy
import pyspark
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

In [2]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

dispensed_database_name = dxpy.find_one_data_object(
    classname="database", name="app*", folder="/", name_mode="glob", describe=True
)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)["id"]

dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset["participant"]


def fields_for_id(field_id):

    field_id = str(field_id)
    fields = participant.find_fields(
        name_regex=r"^p{}(_i\d+)?(_a\d+)?$".format(field_id)
    )

    return sorted(fields, key=lambda f: LooseVersion(f.name))

In [3]:
fields = [
    "21022",
    "22001",
    "22009",
]
field_names = [fields_for_id(id) for id in fields]
field_names = ["eid"] + [field.name for fields in field_names for field in fields]

pcs = {f"p22009_a{i}": f"PC{i}" for i in range(1, 21)}
covs = ["FID", "IID", "SEX", "AGE", "AGE2", "AGESEX", "AGE2SEX"] + list(pcs.values())

In [5]:
df = participant.retrieve_fields(
    names=field_names, engine=dxdata.connect(), coding_values="raw"
)

df = df.na.drop(how="any")

df = (
    df.select([F.col(c).alias(pcs.get(c, c)) for c in df.columns])
    .withColumn("FID", F.col("eid"))
    .withColumn("IID", F.col("eid"))
    .withColumn("SEX", F.col("p22001").cast(IntegerType()))
    .withColumn("AGE", F.col("p21022").cast(IntegerType()))
    .withColumn("AGE2", (F.col("p21022") ** 2).cast(IntegerType()))
    .withColumn("AGESEX", (F.col("p21022") * F.col("p22001")).cast(IntegerType()))
    .withColumn(
        "AGE2SEX", ((F.col("p21022") ** 2) * F.col("p22001")).cast(IntegerType())
    )
    .select(*covs)
)

df.show(5, truncate=False)

[dxdata.dataset.dataset] WARNING: Dataframe schema name(s): ['participant_0001$eid', 'participant_0042$p21022', 'participant_0043$p22001', 'participant_0043$p22009_a1', 'participant_0043$p22009_a2', 'participant_0043$p22009_a3', 'participant_0043$p22009_a4', 'participant_0043$p22009_a5', 'participant_0043$p22009_a6', 'participant_0043$p22009_a7', 'participant_0043$p22009_a8', 'participant_0043$p22009_a9', 'participant_0043$p22009_a10', 'participant_0043$p22009_a11', 'participant_0043$p22009_a12', 'participant_0043$p22009_a13', 'participant_0043$p22009_a14', 'participant_0043$p22009_a15', 'participant_0043$p22009_a16', 'participant_0043$p22009_a17', 'participant_0043$p22009_a18', 'participant_0043$p22009_a19', 'participant_0043$p22009_a20', 'participant_0043$p22009_a21', 'participant_0043$p22009_a22', 'participant_0043$p22009_a23', 'participant_0043$p22009_a24', 'participant_0043$p22009_a25', 'participant_0043$p22009_a26', 'participant_0043$p22009_a27', 'participant_0043$p22009_a28', 'p

+-------+-------+---+---+----+------+-------+--------+-------+--------+--------+---------+---------+---------+---------+---------+----------+--------+---------+---------+---------+---------+---------+---------+----------+-------+----------+
|FID    |IID    |SEX|AGE|AGE2|AGESEX|AGE2SEX|PC1     |PC2    |PC3     |PC4     |PC5      |PC6      |PC7      |PC8      |PC9      |PC10      |PC11    |PC12     |PC13     |PC14     |PC15     |PC16     |PC17     |PC18      |PC19   |PC20      |
+-------+-------+---+---+----+------+-------+--------+-------+--------+--------+---------+---------+---------+---------+---------+----------+--------+---------+---------+---------+---------+---------+---------+----------+-------+----------+
|1000146|1000146|0  |52 |2704|0     |0      |-12.417 |6.75787|-4.42069|0.749104|-1.30339 |0.0162366|1.29456  |-1.45318 |-2.0664  |-2.42804  |-2.04608|-0.119549|-0.705609|-1.9216  |1.42809  |3.75853  |0.0576762|-0.0193767|1.18116|-1.58071  |
|1000181|1000181|0  |62 |3844|0     

In [7]:
df.coalesce(1).write.csv(
    "/tmp/covariates.tsv",
    sep="\t",
    header=True,
)

In [8]:
subprocess.run(
    ["hadoop", "fs", "-getmerge", "/tmp/covariates.tsv", "../tmp/covariates.tsv"],
    check=True,
    shell=False,
)
subprocess.run(
    ["dx", "upload", "../tmp/covariates.tsv", "--path", "/Data/phenotypes/"],
    check=True,
    shell=False,
)

CompletedProcess(args=['dx', 'upload', '../tmp/covariates.tsv', '--path', '/Data/phenotypes/'], returncode=0)